# Real effective exchange rates

Computes the weights matrices for real effective exchange rates following the methodologies of Bems and Johnson (2017) and Patel, Wang, and Wei (2019). REERs are then computed using the weights matrices and MRIO price deflators. Results are saved in `data/reer/`.

## Set up

In [1]:
import numpy as np
import pandas as pd
import duckdb

In [2]:
deflators = 'deflators62.parquet'
weights = 'reer62-weights'
output = 'reer62.parquet'

In [3]:
df1 = duckdb.sql(f"SELECT * FROM read_parquet('../data/reer/{deflators}') WHERE inflation IS NOT NULL").df()
df2 = duckdb.sql(f"SELECT * FROM read_parquet('../data/reer/{weights}.parquet')").df()

years = df1['t'].unique()
G = df1['s'].max()
N = df1['i'].max()
methods = df2['method'].unique()

## Compute REER index

### Country level

In [4]:
reer_agg = pd.DataFrame()

for method in methods:
    
    for year in years:
        
        weights_mt = duckdb.sql(
            f"""
            SELECT * EXCLUDE(method, t)
            FROM read_parquet('../data/reer/{weights}.parquet')
            WHERE method='{method}' AND t={year}
            """
        ).df()

        deflators_t = duckdb.sql(
            f"""
            SELECT * 
            FROM read_parquet('../data/reer/{deflators}') 
            WHERE agg=0 AND t={year}
            """
        ).df()

        reer_t = pd.DataFrame({
            'method': method,
            't': int(year), 's': np.arange(1, G+1), 
            'agg': 0, 'i': 0,
            'reer': weights_mt @ np.array(deflators_t['inflation'])
        })

        reer_agg = pd.concat([reer_agg, reer_t], ignore_index=True)

### Sector level

In [5]:
reer_sector = pd.DataFrame()

for year in years:
    
    weights_t = duckdb.sql(
        f"""
        SELECT * EXCLUDE(t)
        FROM read_parquet('../data/reer/{weights}-sector.parquet')
        WHERE t={year}
        """
    ).df()

    deflators_t = duckdb.sql(
        f"""
        SELECT * 
        FROM read_parquet('../data/reer/{deflators}') 
        WHERE agg=35 AND t={year}
        """
    ).df()
    
    reer_t = pd.DataFrame({
        'method': 'pww-sector',
        't': year, 's': np.arange(1, G+1).repeat(N),
        'agg': 35, 'i': np.tile(np.arange(1, N+1), G),
        'reer': weights_t @ np.array(deflators_t['inflation'])
    })

    reer_sector = pd.concat([reer_sector, reer_t], ignore_index=True)

In [6]:
reer = pd.concat([reer_agg, reer_sector]).dropna().reset_index(drop=True)
reer.to_parquet(f'../data/reer/{output}', index=False)

### View results

In [7]:
duckdb.sql(f"SELECT * FROM read_parquet('../data/reer/{output}')").df()

,method,t,s,agg,i,reer
0,conventional,2007,1,0,0,0.073512
1,conventional,2007,2,0,0,0.003320
2,conventional,2007,3,0,0,0.012165
3,conventional,2007,4,0,0,0.056238
4,conventional,2007,5,0,0,0.036892
...,...,...,...,...,...,...
35905,pww-sector,2021,63,35,31,-0.015792
35906,pww-sector,2021,63,35,32,-0.015512
35907,pww-sector,2021,63,35,33,-0.015191
35908,pww-sector,2021,63,35,34,-0.014267
